In [1]:
import pandas as pd
import re
import requests

from time import sleep
from bs4 import BeautifulSoup

In [2]:
# Extraindo dados do primeiro forum escolhido

def extract_text(text, pattern):
    try:
        found = re.search(pattern, text).group(1)
    except AttributeError:
        found = '0'
    return found

def extract_post(link):
    url  = "https://forumjuridico.org/" + link
    page = requests.get(url, headers={"User-Agent": "XY"})
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find("blockquote", class_="messageText ugc baseHtml").text
    #sleep(1.0)
    return results

# Bloco onde definiremos a quantidade de paginas que vamos buscar dentro do fórum (cada pagina possui 20 perguntas)
# e extraímos as informacoes mais basicas de cada uma (titulo, data, # de visitas, # de respostas e link para a postagem).

areas = ['direito-do-trabalho.17', 'direito-de-familia.11', 'direito-administrativo.15',
        'direito-civil-empresarial-e-do-consumidor.10', 'direito-penal-e-processo-penal.12']

title_lst = []
date_lst = []
answer_lst = []
visit_lst = []
url_lst = []
post_lst = []
area_lst = []
page_old = None

for area in areas:
    
    for i in range(0, 100):
        
        url  = "https://forumjuridico.org/forums/" + area +"/page-" + str(i) + "?_params=Array"
        page = requests.get(url, headers={"User-Agent": "XY"})
             
        # Sair do loop caso cheguemos na ultima pagina de uma determinada area no forum.
        # Podemos fazer isso pois ao chegar na ultima pagina, se aumentaros o valor de i 
        # a ultima pagina continuará sendo mostrada
        
        if page = page_old:
            break
        
        # Request para extrair o documento html
        
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find(id="content")
        elements = soup.find_all("li", class_="discussionListItem visible")
        page_old = page
        
        # Extração das informações pertinentes dentro de cada pagina

        for element in elements:
            title_lst.append(element.find("h3", class_="title").text.strip())
            try:
                date_lst.append(element.find("span", class_="DateTime").text.strip())
            except AttributeError:
                # A pagina de direito de familia utiliza um atributo diferente para o div, entao tratamos o erro aqui
                # para buscar o atributo correto
                date_lst.append(element.find("abbr", class_="DateTime").text.strip())
                
            answer_lst.append(extract_text(element.find("dl", class_="major").text.strip(), '.+?(\d)'))
            visit_lst.append(extract_text(element.find("dl", class_="minor").text.strip(), '.+?(\d)'))
            url_lst.append(extract_text(str(element.find("h3", class_="title").find("a", class_="PreviewTooltip")), 
                                       'href=\"(.+?)\"\ title'))
            post_lst.append(extract_post(extract_text(str(element.find("h3", class_="title").find("a", class_="PreviewTooltip")), 
                                       'href=\"(.+?)\"\ title')))
            area_lst.append(area)

    # Caso seja a primeira area, criar o data frame. Caso contrario, realizar o append
    
    if area = 'direito-do-trabalho.17':
        posts_df = pd.DataFrame(
            {'title': title_lst,
             'date': date_lst,
             'answers': answer_lst,
             'visits': visit_lst,
             'url': url_lst,
             'post': post_lst,
             'area': area_lst
            })
    elif:
        posts_df = posts_df.append(
            pd.DataFrame(
            {'title': title_lst,
             'date': date_lst,
             'answers': answer_lst,
             'visits': visit_lst,
             'url': url_lst,
             'post': post_lst,
             'area': area_lst
            })
        )

posts_df.head()

,title,date,answers,visits,url,post,area
0,recurso cabível exceção de preexecutividade re...,11 de Março de 2021,1,1,threads/recurso-cabivel-excecao-de-preexecutiv...,"\n\t\t\t\t\t\n\t\t\t\t\tBoas tardes, caros col...",direito-do-trabalho.17
1,Liquidez - Reforma Trabalhista,06 de Março de 2021,0,6,threads/liquidez-reforma-trabalhista.23092/,"\n\t\t\t\t\t\n\t\t\t\t\tBoa noite, colegas.\n\...",direito-do-trabalho.17
2,Rito Sumário - sentença irrecorrível?,06 de Março de 2021,0,6,threads/rito-sumario-sentenca-irrecorrivel.23091/,"\n\t\t\t\t\t\n\t\t\t\t\tOlá, colegas.\n\nAlgué...",direito-do-trabalho.17
3,DISPENSA DE FUNCIONARIO AFASTADO EM ESPECIE DE...,27 de Janeiro de 2021,0,9,threads/dispensa-de-funcionario-afastado-em-es...,"\n\t\t\t\t\t\n\t\t\t\t\tPrezados, Drs.\n\nPrim...",direito-do-trabalho.17
4,Cálculo Trabalhista - contador ou advogado?,03 de Maio de 2018,8,1,threads/calculo-trabalhista-contador-ou-advoga...,"\n\t\t\t\t\t\n\t\t\t\t\tQueridos (as),\n\nUm c...",direito-do-trabalho.17


In [13]:
# Extraindo dados de direito empresarial

def extract_text(text, pattern):
    try:
        found = re.search(pattern, text).group(1)
    except AttributeError:
        found = 0
    return found

def extract_post(link):
    url  = "https://www.perguntedireito.com.br" + link
    page = requests.get(url, headers={"User-Agent": "XY"})
    soup = BeautifulSoup(page.content, "html.parser")
    results = extract_text(str(soup), 'itemprop\=\"text\"\>(.+?)\<\/div')
    #sleep(1.0)
    return results

# Bloco onde definiremos a quantidade de paginas que vamos buscar dentro do fórum (cada pagina possui 10 perguntas)
# e extraímos as informacoes mais basicas de cada uma (titulo, data, # de visitas, # de respostas e link para a postagem).

for i in range(0, 20):
    
    url  = "https://www.perguntedireito.com.br/questions/direito-empresarial?start=" + str(i * 10)
    page = requests.get(url, headers={"User-Agent": "XY"})

    # Request para extrair o documento html

    soup = BeautifulSoup(page.content, "html.parser")
    elements = soup.find_all("div", class_="qa-q-list-item")

    title_lst = []
    date_lst = []
    answer_lst = []
    url_lst = []
    post_lst = []

    for element in elements:
        title_lst.append(element.find("div", class_="qa-q-item-title").text.strip())
        date_lst.append(element.find("span", class_="qa-q-item-when-data").text.strip())
        answer_lst.append(extract_text(element.find("span", class_="qa-a-count-data").text.strip(), '.+?(\d)'))
        url_lst.append(extract_text(str(element.find("div", class_="qa-q-item-title")), 
                                   'href=\"(.+?)\"\>'))
        post_lst.append(
            extract_post(
                extract_text(
                    str(
                        element.find("div", class_="qa-q-item-title")
                    ),'href=\"(.+?)\"\>')
            )
        )

# Transformando as listas em um dataframe e visualizando o resultado
     
empresarial_df = pd.DataFrame(
    {'title': title_lst,
     'date': date_lst,
     'answers': answer_lst,
     'url': url_lst,
     'post': post_lst,
     'area': 'empresarial'
    })

empresarial_df.head()

,title,date,answers,url,post,area
0,Qual é a definição de empresa?,em 31 de março de 2013,0,/354/qual-e-a-definicao-de-empresa,"Carolina, a sua resposta foi publicada como co...",empresarial
1,Existem vedações legais ao exercício de empresa?,em 24 de março de 2013,0,/355/existem-vedacoes-legais-ao-exercicio-de-e...,0,empresarial
2,Qual é a diferença entre o valor de negociação...,em 5 de março de 2013,0,/286/diferenca-entre-negociacao-valor-economic...,"Pedro, deixe outras pessoas tentarem responder...",empresarial
3,Qual é a diferença entre o valor nominal e o v...,em 5 de março de 2013,0,/284/qual-diferenca-entre-valor-nominal-valor-...,0,empresarial
4,Sócio declarado incapaz pode permanecer em soc...,em 27 de fevereiro de 2013,0,/198/socio-declarado-incapaz-permanecer-socied...,0,empresarial


In [14]:
# Salvandos os dataframes em um arquivo csv

posts_df.to_csv('dados\posts.csv', sep='|', header=True, index=False)
empresarial_df.to_csv('dados\posts_2.csv', sep='|', header=True, index=False)